In [1]:
import subprocess
import numpy as np
import math
import matplotlib.pyplot as plt
# Enable interactive plot
%matplotlib notebook
import os
from numpy import linalg as la
import cmath
import random
import sys
import pickle
from scipy.optimize import minimize
from sympy import solve, sqrt, exp
import pandas as pd
import gc
gc.collect()

0

# Environmental Fitness Function

In [2]:
# Environmental fitness function with fixed area
def Fitness(x,x0,s):
    #fitness of x in env (x0,s)
    return np.exp(-(x-x0)**2/(2*s**2))/np.sqrt(2*np.pi*s**2)

# Solver for Optimal Phenotype

In [3]:
# Function to check whether phenotype z is the optimal phenotype. Returns zero when the slopes of the adaptive function
# and the fitness sets match, nonzero value otherwise
def Roots(z,x1,s1,x2,s2,p):
    x = z[0]   
    f = np.empty((1))
    f[0] = p*(x-x1)*s2**2*Fitness(x,x1,s1)+(1-p)*(x-x2)*s1**2*Fitness(x,x2,s2)
    return f


# Solver that returns a matrix with optimal phenotype values as a function of time t in range(0,T), for a given rate of
# environmental change v, and for NP different p values. Also returns the time at which the optimal phenotype switches 
# from a generalist to a specialist
def optp(v,T):    
    optimalmat=np.zeros((NP,T))
    arch1=np.zeros(T)
    arch2=np.zeros(T)    
    specialisttime=np.zeros((NP))
    times=np.arange(0,T,1)
    opt=(x01+x02)/2

    for n in range(NP):
        p=p_range[n]        
        specialistcheck=0
        for t in range(T):
            x1=x01
            x2=x02+v*t         
            if(n==0):
                arch1[t]=x1
                arch2[t]=x2

            # The solver is sensitive to initial guesses, so we compare two solutions: one (zGuess1) where the initial guess
            # is one of the specialists, and another (zGuess2) where the initial guess is the optimal solution in the
            # previous time step. This produces the best results, especially for skewed fitness sets.

            m=(Fitness(x2,x2,s2)-Fitness(x1,x2,s2))/(Fitness(x2,x1,s1)-Fitness(x1,x1,s1)) #slope of line joining the two specialists on the fitness set
            mp=-p/(1-p)    #slope of the adaptive function
            if(mp<=m):
                zGuess1 = np.array([x1])         # if the slope of the adaptive function is smaller that that of the 
            elif(mp>m):                          # line joining the two specialists on the fitness set, choose x1 as the initial guess, 
                zGuess1 = np.array([x2])         # otherwise choose x2.
            
            bounds = [(x1,x2)]                   # only look for solutions in the range(x1,x2)
            
            # Roots(z,x1,s1,x2,s2,p) returns a value f for each z, the following finds z within the defined bounds 
            # that minimizes |f|, starting with some initial guesses for z..
            z1 = minimize(lambda z: np.linalg.norm(Roots(z,x1,s1,x2,s2,p)),tol=1e-15, x0=zGuess1, bounds=bounds)
            opt1=z1.x[0]
            A1=p*Fitness(opt1,x1,s1)+(1-p)*Fitness(opt1,x2,s2) # value of Adaptive function at the first solution, opt1
            
            #repeat for second initial guess
            zGuess2 = np.array([opt])
            bounds = [(x1,x2)]
            z2 = minimize(lambda z: np.linalg.norm(Roots(z,x1,s1,x2,s2,p)),tol=1e-15, x0=zGuess2, bounds=bounds)
            opt2=z2.x[0]
            A2=p*Fitness(opt2,x1,s1)+(1-p)*Fitness(opt2,x2,s2)  # value of Adaptive function at the second solution, opt2

            if(A2>A1):         # chose opt2 as the correct optimal if A2>A1, else choose opt1
                opt=opt2
            else:
                opt=opt1
                
            optimalmat[n,t]=opt
            
            if specialistcheck==0 and (abs(opt-x1)<0.5 or abs(opt-x2)<0.5): # check if the optimal has collapsed on to one of
                specialisttime[n]=t                                         # the specialists, and record time t as the time 
                specialistcheck=1                                           # of the switch
                 
        if specialistcheck==0:
            specialisttime[n]=np.inf
    return [optimalmat, specialisttime]

# Tolerances for The Effective Fitness Function

In [4]:
# Function that returns the tolerance of the effective fitness function that governs selection as a function of time,
# for a given rate of environmental change, and for NP different p values.
def opts(v,T,optimalmat):
    sigmamat=np.zeros((NP,T))
    for n in range(NP):
        for t in range(T):
            if abs(x01-optimalmat[n,t])<0.5:
                sigmamat[n,t]=s1
            elif abs(x02+v*t-optimalmat[n,t])<0.5:
                sigmamat[n,t]=s2
            else:
                sigmamat[n,t]=sqrt(p_range[n]*s1**2+(1-p_range[n])*s2**2)
    return sigmamat

# Population Dynamics: Selection, Mutation & Resizing for Carrying Capacity

In [5]:
# Population level dynamics in which selection is governed by an effective fitness function which has a maxima at the
# optimal phenotype, and a tolerance obtained from a weighted sum of the two environmental tolerances. Returns 
# the following information as a function of time for a given rate of environmental change v, and NP different p 
# values for a single population: population size, average phenotype, variance of the population, whether the 
# population survives, and extinction time
def dynamics_new(v,T,optimalmat,sigmamat):      
    popmean=np.zeros((NP,T))
    popsize=np.zeros((NP,T))
    popdev=np.zeros((NP,T))
    popsurvival=np.zeros(NP,dtype=int)
    extincttime=np.zeros(NP)    
    for l in range(NP):    
        seff=math.sqrt(p_range[l]*s1**2+(1-p_range[l])*s2**2)          # tolerance of effective fitness function
        sigmapop=math.sqrt(mutr**2+mutr*math.sqrt(mutr**2+4*seff**2))/2     # variance of initial phenotypic distribution
        mu, sigma = optimalmat[l,0],sigmapop                           # mean and standard deviation of the initial phenotypic distribution
        s = np.random.normal(mu, sigma, initsize)                      # initialize the population
        
        # Turn the set of phenotypes into a histogram distribution
        binnumber=30  
        dist=np.zeros((binnumber,2))
        hist,bin_edges=np.histogram(s,bins=binnumber)
        dist[:,0]=(bin_edges[1:]+bin_edges[:-1])/2
        dist[:,1]=hist    

        for time in range(T):
            newpheno=[]
            optimal=optimalmat[l,time]
            omegaf=sigmamat[l,time]
            
            # For each phenotype, compute the number of offsprings in the next generation after selection by
            # an effective fitness function, and then add mutations as Gaussian noise with mean zero and variance
            # equal to the mutation rate mutr            
            for i in range(binnumber): 
                f=math.floor(dist[i,1]*Fmax*math.exp(-((dist[i,0]-(optimal))**2)/(2*omegaf**2))) 
                for p in range (f):
                    newpheno.append(random.gauss(dist[i,0], mutr))  
                    
            newpheno=np.array(newpheno)   
            size=len(newpheno)
            popsize[l,time]=size
            if time==T-1:                     # Population survives if its size does not fall below 5 till t=T
                popsurvival[l]=1
                extincttime[l]=np.inf
            if size<=5:                       # Population does not survive if its size falls below 5 before t=T
                popmean[l,time:]=0            # breaks out of the dynamics simulation
                popdev[l,time:]=0
                popsurvival[l]=0
                extincttime[l]=time
                break    
            elif size>5:
                popmean[l,time]=np.average(newpheno)
                popdev[l,time]=np.std(newpheno)
            
            if size>initsize:                                             # if population size is greater than 
                newphenosized=random.sample(list(newpheno), initsize)     # carrying capacity K, pick a random  
            else:                                                         # sample of K individuals for to undergo
                newphenosized=newpheno                                    # selection in the next generation

            hist,bin_edges=np.histogram(newphenosized,bins=binnumber)     # Turn the resized population after selection  
            dist[:,0]=(bin_edges[1:]+bin_edges[:-1])/2                    # and added mutations into a histogram distribution
            dist[:,1]=hist                                                # that undergoes selection in the next generation
   
    return [popsize, popmean, popdev, popsurvival, extincttime]


#### Obtain Optimal Phenotype, Tolerance for Effective Fitness Function & Time of Switch to Specialist Phenotype

In [ ]:
Nv=31
v0=0.05
dv=0.02
v_range=np.arange(v0,v0+Nv*dv,dv)    # range of rates of environmental change v to study
print(v_range)

T_range=np.zeros((Nv),dtype=int)     # range of simulation durations for different v values

Ns=31
sr0=sr=0.5
ds=0.05
s_range=np.arange(sr0,sr0+Ns*ds,ds)  # range of tolerance asymmetry ratios s2/s1 for the two environments
print(s_range)

NP=6   #number of p values we will use
p_range=np.array([0.105,0.205,0.305,0.405,0.505,0.605])  # range of values for the frequency p with which the stationary environment is encountered
print(p_range)

s1=10                  # Tolerance of the stationary environment
s2=0                   # Initialize the tolerance of the moving environment
seff=0                 # Initialize the variance of the effective fitness function that governs selection
sigmapop=0             # Initialize the variance of initial phenotypic distribution
T0=T=1200              # Maximum simulation duration
x01=100                # Initial phenotypic value of stationary specialist
x02=110                # Initial phenotypic value of moving specialist
initsize=1000          # carrying capacity K
mutr=0.5               # mutation rate
Fmax=2                 # maximum number of offsprings

Niter=1               # Number of populations to study for each v,p and s2/s1

optimalmat=np.zeros((Nv,Ns,NP,T0))
sigmamat=np.zeros((Nv,Ns,NP,T0))
specialisttime=np.zeros((Nv, Ns, NP))

dir1="DATAPOP_mutr{}/".format(mutr)
if not os.path.exists(dir1):
    os.makedirs(dir1)

for i in range(Nv):
    v=v_range[i]
    T_range[i]=T=int(T0*v0/v)    # Fix simulation duration depending on v, when the second environment moves faster,
                                 # the switch from generalist to a specialist optimal phenotype occurs sooner.
    for j in range(Ns):
        sr=s_range[j]
        s2=sr*s1                 # Fix the tolerance of the moving environment
       
        [optimalmat[i,j,:,:T],specialisttime[i,j]]=optp(v,T)    # calculate time of switch from a generalist to a specialist
                                                                # optimal phenotype, as well as the optimal phenotype as a 
                                                                # function of time, for NP different p values, and for a 
                                                                # given value of v and s2/s1
        print(v,sr,T,specialisttime[i,j],optimalmat[i,j,:,T-1])
        sigmamat[i,j,:,:T]=opts(v,T,optimalmat[i,j,:,:T])       # calculate the tolerances for the effective fitness function
                                                                # governing selection as a function of time for NP different p 
                                                                # values, and for a given value of v and s2/s1
        
pickle.dump(s_range,open(dir1+'s_range.p','wb'))
pickle.dump(v_range,open(dir1+'v_range.p','wb'))
pickle.dump(T_range,open(dir1+'T_range.p','wb'))
pickle.dump(p_range,open(dir1+'p_range.p','wb'))
pickle.dump(optimalmat,open(dir1+'optimalmat.p','wb'))
pickle.dump(sigmamat,open(dir1+'sigmamat.p','wb'))
pickle.dump(specialisttime,open(dir1+'specialisttime.p','wb'))

[0.05 0.07 0.09 0.11 0.13 0.15 0.17 0.19 0.21 0.23 0.25 0.27 0.29 0.31
 0.33 0.35 0.37 0.39 0.41 0.43 0.45 0.47 0.49 0.51 0.53 0.55 0.57 0.59
 0.61 0.63 0.65]
[0.5  0.55 0.6  0.65 0.7  0.75 0.8  0.85 0.9  0.95 1.   1.05 1.1  1.15
 1.2  1.25 1.3  1.35 1.4  1.45 1.5  1.55 1.6  1.65 1.7  1.75 1.8  1.85
 1.9  1.95 2.  ]
[0.105 0.205 0.305 0.405 0.505 0.605]
0.05 0.5 1200 [  0.   0.   0.  50. 155. 216.] [169.95 169.95 169.95 169.95 169.95 169.95]
0.05 0.55 1200 [  0.   0.   0. 133. 200. 251.] [169.95 169.95 169.95 169.95 169.95 169.95]
0.05 0.6000000000000001 1200 [  0.   0.  90. 178. 233. 279.] [169.95 169.95 169.95 169.95 169.95 169.95]
0.05 0.6500000000000001 1200 [  0.   0. 145. 212. 261. 303.] [169.95 169.95 169.95 169.95 169.95 169.95]
0.05 0.7000000000000002 1200 [  0.  67. 182. 240. 285. 270.] [169.95 169.95 169.95 169.95 169.95 100.  ]
0.05 0.7500000000000002 1200 [  0. 124. 211. 264. 305. 246.] [169.95 169.95 169.95 169.95 169.95 100.  ]
0.05 0.8000000000000003 1200 [  0. 161. 236

0.09000000000000001 0.5 666 [  0.   0.   0.  28.  86. 120.] [169.85 169.85 169.85 169.85 169.85 169.85]
0.09000000000000001 0.55 666 [  0.   0.   0.  74. 111. 139.] [169.85 169.85 169.85 169.85 169.85 169.85]
0.09000000000000001 0.6000000000000001 666 [  0.   0.  50.  99. 130. 155.] [169.85 169.85 169.85 169.85 169.85 169.85]
0.09000000000000001 0.6500000000000001 666 [  0.   0.  81. 118. 145. 169.] [169.85 169.85 169.85 169.85 169.85 169.85]
0.09000000000000001 0.7000000000000002 666 [  0.  37. 101. 134. 158. 150.] [169.85 169.85 169.85 169.85 169.85 100.  ]
0.09000000000000001 0.7500000000000002 666 [  0.  69. 117. 147. 170. 137.] [169.85 169.85 169.85 169.85 169.85 100.  ]
0.09000000000000001 0.8000000000000003 666 [  0.  90. 131. 158. 180. 148.] [169.85 169.85 169.85 169.85 169.85 100.  ]
0.09000000000000001 0.8500000000000003 666 [  0. 105. 143. 168. 189. 160.] [169.85 169.85 169.85 169.85 169.85 100.  ]
0.09000000000000001 0.9000000000000004 666 [ 28. 119. 153. 177. 197. 170.] [1

0.11000000000000001 1.9000000000000012 545 [164. 192. 246. 337. 300. 257.] [169.84       169.84       169.84       100.01747011 100.01164377
 100.0077503 ]
0.11000000000000001 1.9500000000000013 545 [167. 195. 266. 343. 305. 260.] [169.84       169.84       169.84       100.02276665 100.01517022
 100.01009593]
0.11000000000000001 2.0000000000000013 545 [170. 197. 289. 350. 310. 262.] [169.84       169.84       169.84       100.02898597 100.01930925
 100.01284822]
0.13 0.5 461 [ 0.  0.  0. 20. 60. 83.] [169.8 169.8 169.8 169.8 169.8 169.8]
0.13 0.55 461 [ 0.  0.  0. 51. 77. 97.] [169.8 169.8 169.8 169.8 169.8 169.8]
0.13 0.6000000000000001 461 [  0.   0.  35.  69.  90. 108.] [169.8 169.8 169.8 169.8 169.8 169.8]
0.13 0.6500000000000001 461 [  0.   0.  56.  82. 101. 117.] [169.8 169.8 169.8 169.8 169.8 169.8]
0.13 0.7000000000000002 461 [  0.  26.  70.  93. 110. 104.] [169.8 169.8 169.8 169.8 169.8 100. ]
0.13 0.7500000000000002 461 [  0.  48.  81. 102. 118.  95.] [169.8 169.8 169.8 169.

In [ ]:
# Nv=31
# v0=0.05
# Ns=31
# s1=10
# s2=seff=sigmapop=0
# T0=T=1200
# NP=6
# x01=100
# x02=110
# initsize=1000
# mutr=0.5 #mutation rate
# Fmax=2 # maximum fecundity

# dir1="data_mutr{}/".format(mutr)
# if not os.path.exists(dir1):
#     os.mkdir(dir1)
    
# v_range=pickle.load(open(dir1+'v_range.p','rb'))
# s_range=pickle.load(open(dir1+'s_range.p','rb'))
# T_range=pickle.load(open(dir1+'T_range.p','rb'))
# p_range=pickle.load(open(dir1+'p_range.p','rb'))
# optimalmat=pickle.load(open(dir1+'optimalmat.p','rb'))
# sigmamat=pickle.load(open(dir1+'sigmamat.p','rb'))
# specialisttime=pickle.load(open(dir1+'specialisttime.p','rb'))
# print(v_range)
# print(T_range)
# print(s_range)
# print(p_range)

### Run Population Level Simulations

In [ ]:
survival=np.zeros((Niter, Nv, Ns, NP))
meanpop=np.zeros((Niter, Nv, Ns, NP, T))
extincttime=np.zeros((Niter, Nv, Ns, NP))
popsize=np.zeros((Niter, Nv, Ns, NP,T))
popdev=np.zeros((Niter, Nv, Ns, NP,T))
start=15
for r in range(Niter): 
    dir2=dir1+'Pop{}/'.format(start+r+1)
    if not os.path.exists(dir2):
        os.mkdir(dir2)
    
    print("--------------------- Population {} --------------------".format(start+r+1))
    f1=open(dir2+"survival.txt","w")
    f2=open(dir2+"meanpop.txt","w")
    f3=open(dir2+"extinct_time.txt","w")
    f4=open(dir2+"popsize.txt","w")
    f5=open(dir2+"popdev.txt","w")
    for i in range(Nv):
        v=v_range[i]
        T=T_range[i]    
        for j in range(Ns):
            sr=s_range[j]
            s2=sr*s1            
            print('v = {}, T = {}, sr = {}'.format(v,T,sr))            
            f1.write("{:.2f} {:.2f} ".format(v_range[i],s_range[j]))
            f3.write("{:.2f} {:.2f} ".format(v_range[i],s_range[j]))                 

            [size,mean,deviation,survive,extinct]=dynamics_new(v,T,optimalmat[i,j,:,:T],sigmamat[i,j,:,:T]) 
   
            
            survival[r,i,j]=survive
            extincttime[r,i,j]=extinct
            meanpop[r,i,j,:,:T]=mean   
            popsize[r,i,j,:,:T]=size
            popdev[r,i,j,:,:T]=deviation
            for k in range(NP):
                f2.write("{:.2f} {:.2f} {} ".format(v_range[i],s_range[j],p_range[k]))
                f4.write("{:.2f} {:.2f} {} ".format(v_range[i],s_range[j],p_range[k]))
                f5.write("{:.2f} {:.2f} {} ".format(v_range[i],s_range[j],p_range[k]))
                for t in range(T):
                    f2.write("{} ".format(meanpop[r,i,j,k,t]))
                    f4.write("{} ".format(popsize[r,i,j,k,t]))
                    f5.write("{} ".format(popdev[r,i,j,k,t]))
                f2.write("\n")
                f2.flush()
                f4.write("\n")
                f4.flush()
                f5.write("\n")
                f5.flush()
                
                f1.write("{} ".format(survival[r,i,j,k]))
                f3.write("{} ".format(extincttime[r,i,j,k]))
            f1.write("\n")
            f1.flush()
            f3.write("\n")
            f3.flush()
       
    f1.close()
    f2.close()
    f3.close()
    f4.close()
    f5.close()